In [1]:
import time
from datetime import datetime, timedelta
import ciso8601
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pmaw import PushshiftAPI 
from reddit_tools import clean_reddit_data

# https://github.com/asad70/reddit-sentiment-analysis

api = PushshiftAPI()


{}


In [2]:
subreddit="cryptocurrency"

In [51]:
# ini_date refers to the first date to be scraped from going back in time. end_date is the last date to be scraped.
# TODO either change the loop to scrape from day 0 of each year or account for leap years with unix timestamps

def get_annualized_submissions(subreddit, end_date, ini_date = None):

    unix_year = 31536000
    limit =  None
    
    if ini_date is None:
        ini_date_object = datetime.today()


    end_date_object = ciso8601.parse_datetime(end_date)
    first_day_e = datetime(end_date_object.year, 1,1)
    first_day_e_epoch = int(time.mktime(first_day_e.timetuple()))



    first_day_i = datetime(ini_date_object.year, 1,1)
    first_day_i_epoch = int(time.mktime(first_day_i.timetuple()))



    n_loops = (ini_date_object - end_date_object).days / 365

    if n_loops is not int:
        n_loops = int(n_loops) + 1


    end_date_epoch = int(time.mktime(end_date_object.timetuple()))
    ini_date_epoch = int(time.mktime(ini_date_object.timetuple()))

    add_remainder = ini_date_epoch - first_day_i_epoch

    unix_ts = np.flip(np.arange(0, n_loops) * unix_year)
    unix_ts = np.append(unix_ts, unix_ts[-1])
    unix_ts = ini_date_epoch - unix_ts
    unix_ts[:-1] = unix_ts[:-1] - add_remainder

    for i in unix_ts:
        i_date_str = datetime.fromtimestamp(i).strftime('%d-%m-%y')

    unix_ts = unix_ts.astype(int)

    if n_loops == 0:
        print("Range of dates provided is less than one year, only one dataset will be created")
        submissions = api.search_submissions(subreddit=subreddit, limit = limit, before = ini_date_epoch , after = end_date_epoch)
        sub_df = pd.DataFrame(submissions)

        i_date_str = datetime.fromtimestamp(ini_date_epoch).strftime('%d-%m-%y')
        e_date_str = datetime.fromtimestamp(end_date_epoch).strftime('%d-%m-%y') 


        print(f'{len(sub_df)} posts retrieved from Pushshift from {e_date_str} to {i_date_str}')
        
        sub_df.to_csv(f'{subreddit}_submissions_{e_date_str}_{i_date_str}.csv') 

        return sub_df


    for cnt, i  in enumerate(unix_ts[:-1]):
        dt_before = unix_ts[cnt + 1]
        dt_after = i
        
        i_date_str = datetime.fromtimestamp(dt_before).strftime('%d-%m-%y')
        print(i_date_str)
        e_date_str = datetime.fromtimestamp(dt_after).strftime('%d-%m-%y') 
        print(e_date_str)
        
        submissions = api.search_submissions(subreddit=subreddit, limit = limit, before = dt_before , after = dt_after)
        sub_df = pd.DataFrame(submissions)
        print(f'{len(sub_df)} posts retrieved from Pushshift from {e_date_str} to {i_date_str}')
        
        sub_df.to_csv(f'{subreddit}_submissions_{e_date_str}_{i_date_str}.csv') 
        
        if cnt == 0:
            sub_df_all = pd.DataFrame(submissions)
        else:
            sub_df_all = pd.concat([sub_df_all, sub_df])
        
   
        
    i_date_str = datetime.fromtimestamp(ini_date_epoch).strftime('%d-%m-%y')
    e_date_str = datetime.fromtimestamp(first_day_e_epoch).strftime('%d-%m-%y')    

    sub_df_all.to_csv(f'ALL_{subreddit}_submissions_{e_date_str}_{i_date_str}.csv') 

    
    return sub_df_all




In [49]:
from datetime import datetime
before_btc = "2020-01-01"


In [50]:

    
sub_scrape = get_annualized_submissions("cryptocurrency", before_btc)


01-01-21
02-01-20


Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift shards are active. Query results may be incomplete.
Not all PushShift sh

78660 posts retrieved from Pushshift from 02-01-20 to 01-01-21
01-01-22
01-01-21
638873 posts retrieved from Pushshift from 01-01-21 to 01-01-22
14-05-22
01-01-22
132379 posts retrieved from Pushshift from 01-01-22 to 14-05-22


In [2]:
sub_scrape = pd.read_csv("cryptocurrency_submissions_01-01-21_01-01-22.csv", index_col = 0)  



/tmp/ipykernel_655078/3554674894.py:1: DtypeWarning: Columns (77,79,83,85,86,87,89,91,92,93,94,95,96) have mixed types. Specify dtype option on import or set low_memory=False.
  sub_scrape = pd.read_csv("cryptocurrency_submissions_01-01-21_01-01-22.csv", index_col = 0)


In [3]:
clean = clean_reddit_data(sub_scrape)

the dataset initially had 96 columns and 638873 rows
Now we have 13 columns and 354347 rows 
 bit better innit?


In [ ]:

# get columns from fetched data
cols = sub_scrape.columns
print(f"the dataset currently has {cols.shape} columns")
cols

In [71]:
# as expected, the number of variables returned is rather large, making the dataframe hard to work with. some of the variables are 
# explicitly not useful for the analysis taking place now, so we will drop them. others require further assesment to verify their content.

clean_submissions = sub_scrape.drop(columns = ['all_awardings','allow_live_comments','author_flair_css_class', 'author_flair_richtext', 'author_flair_text',
       'author_flair_type', 'author_fullname', 'author_patreon_flair',
       'author_premium', 'awarders', 'can_mod_post', 'link_flair_background_color', 'link_flair_richtext',
       'link_flair_text_color', 'link_flair_type', 'locked','over_18','crosspost_parent', 'crosspost_parent_list', 'link_flair_css_class', 'link_flair_template_id', 'link_flair_text',
       'thumbnail_height', 'thumbnail_width','edited', 'banned_by','author_is_blocked', 'poll_data', 'event_end',
       'event_is_live', 'event_start', 'removal_reason'])


In [72]:
# once we have dropped those variables, it will be easier to visualize the remaining data and assess their relevancy for our task
# To do so, we will get the first row to get a closer look of the data in hand.
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
clean_submissions.tail()

,author,contest_mode,created_utc,domain,full_link,gildings,id,is_crosspostable,is_meta,is_original_content,is_reddit_media_domain,is_robot_indexable,is_self,is_video,media_only,no_follow,num_comments,num_crossposts,parent_whitelist_status,permalink,pinned,pwls,removed_by_category,retrieved_on,score,selftext,send_replies,spoiler,stickied,subreddit,subreddit_id,subreddit_subscribers,subreddit_type,thumbnail,title,total_awards_received,treatment_tags,upvote_ratio,url,whitelist_status,wls,post_hint,preview,url_overridden_by_dest,author_flair_template_id,author_flair_text_color,author_flair_background_color,media_metadata,media,media_embed,secure_media,secure_media_embed,is_gallery,author_cakeday,gallery_data,gilded,discussion_type,suggested_sort,is_created_from_ads_ui,top_awarded_type,tournament_data,distinguished,collections,live_audio
638868,rudolfo1998,False,1621427944,self.CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,{},ng4p1l,True,False,False,False,True,True,False,False,True,10,0,all_ads,/r/CryptoCurrency/comments/ng4p1l/question_fro...,False,6,NaN,1621427957,1,I started investing into cryptos a few weeks b...,True,False,False,CryptoCurrency,t5_2wlj3,2825523,public,self,Question from a newbie in the crypto market,0,[],1.0,https://www.reddit.com/r/CryptoCurrency/commen...,all_ads,6,NaN,NaN,NaN,NaN,dark,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
638869,guidre,False,1621427919,cryptonews.com,https://www.reddit.com/r/CryptoCurrency/commen...,{},ng4oom,False,False,False,False,False,False,False,False,True,0,0,all_ads,/r/CryptoCurrency/comments/ng4oom/finally_a_de...,False,6,moderator,1621427930,1,NaN,True,False,False,CryptoCurrency,t5_2wlj3,2825517,public,https://b.thumbs.redditmedia.com/77rMUZWnsYNeK...,Finally a decent bit of journalism that looks ...,0,[],1.0,https://cryptonews.com/news/another-usual-bitc...,all_ads,6,link,"{'enabled': False, 'images': [{'id': '028yj27m...",https://cryptonews.com/news/another-usual-bitc...,NaN,dark,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
638870,hnr01,False,1621427904,self.CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,{},ng4ohd,True,False,False,False,True,True,False,False,True,30,0,all_ads,/r/CryptoCurrency/comments/ng4ohd/what_are_you...,False,6,NaN,1621427915,1,I hope each of you are learning something abou...,True,False,False,CryptoCurrency,t5_2wlj3,2825513,public,self,What are you learning?,0,[],1.0,https://www.reddit.com/r/CryptoCurrency/commen...,all_ads,6,NaN,NaN,NaN,38521cf8-291e-11e5-aaf3-0ee61c357d3b,dark,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
638871,hboi31,False,1621427885,self.CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,{},ng4o8a,False,False,False,False,False,True,False,False,True,2,0,all_ads,/r/CryptoCurrency/comments/ng4o8a/when_lambo/,False,6,moderator,1621427897,1,[removed],True,False,False,CryptoCurrency,t5_2wlj3,2825514,public,self,When lambo?,0,[],1.0,https://www.reddit.com/r/CryptoCurrency/commen...,all_ads,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
638872,BRICKDJV2,False,1621427849,self.CryptoCurrency,https://www.reddit.com/r/CryptoCurrency/commen...,{},ng4nq0,True,False,False,False,True,True,False,False,True,35,0,all_ads,/r/CryptoCurrency/comments/ng4nq0/dont_panic_a...,False,6,NaN,1621427861,1,So BTC has dropped approximately 44.22% from i...,True,False,False,CryptoCurrency,t5_2wlj3,2825503,public,self,Dont panic! A quick reassurance post with data...,0,[],1.0,https://www.reddit.com/r/CryptoCurrency/commen...,all_ads,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
# since the number of variables containing valuable information is smaller than the number of irrelevant variables, using .drop()  
# would be slower than simply selecting the columns we do need

clean_submissions = clean_submissions[["author", "contest_mode", "created_utc", "domain", "id", 
                                       "num_comments", "num_crossposts", "removed_by_category", 
                                       "score", "selftext", "subreddit", "subreddit_subscribers", 
                                       "title", "total_awards_received", "upvote_ratio"]]

clean_submissions.columns.shape       

(15,)

In [76]:
clean_submissions.head()
# len(clean_submissions) # 638873


,author,contest_mode,created_utc,domain,id,num_comments,num_crossposts,removed_by_category,score,selftext,subreddit,subreddit_subscribers,title,total_awards_received,upvote_ratio
0,JDUNN02,False,1615726743,self.CryptoCurrency,m4uwq8,0,0,moderator,1,[removed],CryptoCurrency,1901145,Selling at top question,0,1.0
1,gatesthree,False,1615726739,i.redd.it,m4uwoe,3,0,NaN,1,NaN,CryptoCurrency,1901145,I'm curious to see what you guys think of this...,0,1.0
2,Flashy_Discount9506,False,1615726730,self.CryptoCurrency,m4uwl4,0,0,moderator,1,[removed],CryptoCurrency,1901144,Can anyone help me with some crypto trouble ou...,0,1.0
3,InexistentKnight,False,1615726719,self.CryptoCurrency,m4uwgx,28,0,NaN,1,I know many believe governments cannot crack d...,CryptoCurrency,1901144,Change my mind: BTC is the riskiest of big cry...,0,1.0
4,System_Little,False,1615726670,self.CryptoCurrency,m4uw10,1,0,moderator,1,[removed],CryptoCurrency,1901136,ETH-Miner on Telegram: scam?,0,1.0


In [77]:
clean_submissions.drop(clean_submissions.index[clean_submissions['contest_mode'] == True], inplace=True)
print(clean_submissions.shape)
clean_submissions.columns

(638799, 15)


Index(['author', 'contest_mode', 'created_utc', 'domain', 'id', 'num_comments', 'num_crossposts', 'removed_by_category', 'score', 'selftext', 'subreddit', 'subreddit_subscribers', 'title', 'total_awards_received', 'upvote_ratio'], dtype='object')

In [78]:
unwanted = ["moderator", "reddit", "automod_filtered", "content_takedown"]
clean_submissions = clean_submissions[~clean_submissions['removed_by_category'].isin(unwanted)]
print(clean_submissions.shape)

clean_submissions = clean_submissions.drop(columns = ['removed_by_category',"contest_mode"])


(354347, 15)


In [79]:

print(clean_submissions.shape)
clean_submissions.tail()

(354347, 13)


,author,created_utc,domain,id,num_comments,num_crossposts,score,selftext,subreddit,subreddit_subscribers,title,total_awards_received,upvote_ratio
638865,HuskerNatChamps2020,1621427957,coindesk.com,ng4p8a,3,0,1,NaN,CryptoCurrency,2825526,Egypt's Largest Bank Joins Ripple Network for ...,0,1.0
638867,itsbuzzpoint,1621427948,coindesk.com,ng4p3l,2,0,1,NaN,CryptoCurrency,2825524,India’s Government May Form Panel to Study Reg...,0,1.0
638868,rudolfo1998,1621427944,self.CryptoCurrency,ng4p1l,10,0,1,I started investing into cryptos a few weeks b...,CryptoCurrency,2825523,Question from a newbie in the crypto market,0,1.0
638870,hnr01,1621427904,self.CryptoCurrency,ng4ohd,30,0,1,I hope each of you are learning something abou...,CryptoCurrency,2825513,What are you learning?,0,1.0
638872,BRICKDJV2,1621427849,self.CryptoCurrency,ng4nq0,35,0,1,So BTC has dropped approximately 44.22% from i...,CryptoCurrency,2825503,Dont panic! A quick reassurance post with data...,0,1.0
